# Project Setup: Tumor Detection and Segmentation with MONAI

This notebook guides you through setting up the directory structure and essential files for a MONAI-based tumor detection and segmentation project.

## 1. Create Directory Structure

Run the following commands in your terminal to create the recommended project structure.

### (Optional) Create and Activate a Virtual Environment

It is recommended to use a Python virtual environment for dependency isolation.

#### If you get a "Permission denied" error when creating directories, use `sudo`:

You may need administrator rights to create directories in some locations.

In [10]:
# Add 'sudo' if you get a permission error (run in terminal)
!sudo mkdir -p tumor-detection-segmentation/data tumor-detection-segmentation/models tumor-detection-segmentation/notebooks tumor-detection-segmentation/src

In [ ]:
# Create and activate a virtual environment (run in terminal)
!python3 -m venv venv
# On Linux/macOS, activate with:
# source venv/bin/activate
# On Windows, activate with:
# .\venv\Scripts\activate

In [12]:
# If you get a permission error, try running with 'sudo' (in terminal, not in Jupyter):
# !sudo mkdir -p tumor-detection-segmentation/data tumor-detection-segmentation/models tumor-detection-segmentation/notebooks tumor-detection-segmentation/src
# !sudo touch tumor-detection-segmentation/src/__init__.py
# !sudo touch tumor-detection-segmentation/src/train.py tumor-detection-segmentation/src/evaluate.py tumor-detection-segmentation/src/inference.py
# !sudo touch tumor-detection-segmentation/src/data_preprocessing.py tumor-detection-segmentation/src/dataset.py tumor-detection-segmentation/src/utils.py
# !sudo touch tumor-detection-segmentation/requirements.txt tumor-detection-segmentation/.gitignore tumor-detection-segmentation/README.md tumor-detection-segmentation/config.json

# If you do not have sudo access or are working in a restricted environment (e.g., JupyterHub), 
# create the project inside your home directory or another location where you have write permissions:
import os

base_dir = os.path.expanduser("~/tumor-detection-segmentation")
os.makedirs(os.path.join(base_dir, "data"), exist_ok=True)
os.makedirs(os.path.join(base_dir, "models"), exist_ok=True)
os.makedirs(os.path.join(base_dir, "notebooks"), exist_ok=True)
os.makedirs(os.path.join(base_dir, "src"), exist_ok=True)

for fname in [
    "src/__init__.py",
    "src/train.py",
    "src/evaluate.py",
    "src/inference.py",
    "src/data_preprocessing.py",
    "src/dataset.py",
    "src/utils.py",
    "requirements.txt",
    ".gitignore",
    "README.md",
    "config.json"
]:
    fpath = os.path.join(base_dir, fname)
    if not os.path.exists(fpath):
        open(fpath, "a").close()

## 2. Add Python Dependencies

List the required Python libraries in `requirements.txt`.

In [13]:
# Write dependencies to requirements.txt
requirements = """monai
torch
torchvision
numpy
matplotlib
pandas
scikit-learn
scipy
tqdm
jupyter
"""
with open("tumor-detection-segmentation/requirements.txt", "w") as f:
    f.write(requirements)

## 3. Configure .gitignore

Add common patterns to ignore unnecessary files and folders.

In [14]:
# Write .gitignore content
gitignore = """# Python
__pycache__/
*.pyc
*.pyo
*.pyd

# Jupyter Notebook checkpoints
.ipynb_checkpoints/

# Virtual environments
venv/
.env/

# Data and models
data/
models/
"""
with open("tumor-detection-segmentation/.gitignore", "w") as f:
    f.write(gitignore)

## 4. Create Project Documentation

Add a `README.md` file to describe the project and setup instructions.

In [15]:
# Write README.md content
readme = """# Tumor Detection and Segmentation using MONAI

This project uses MONAI for detecting and segmenting tumors in MRI or CT images.

## Directory Structure
- `data/`: Store datasets here.
- `models/`: Save trained models here.
- `notebooks/`: Jupyter notebooks for experiments.
- `src/`: Source code for training, evaluation, and inference.

## Setup
1. Clone the repository.
2. Install dependencies:
   ```bash
   pip install -r requirements.txt
   ```
3. Run the training script:
   ```bash
   python src/train.py
   ```
"""
with open("tumor-detection-segmentation/README.md", "w") as f:
    f.write(readme)

## 5. Add Configuration File

Create a `config.json` file with default settings for training and data paths.

In [16]:
# Write config.json content
import json
config = {
  "data_path": "./data",
  "model_save_path": "./models",
  "batch_size": 16,
  "epochs": 50,
  "learning_rate": 0.001,
  "image_size": [128, 128, 128],
  "num_workers": 4,
  "device": "cuda"
}
with open("tumor-detection-segmentation/config.json", "w") as f:
    json.dump(config, f, indent=2)

## 6. Example Training Script

Below is a starter script for training a UNet model using MONAI. Save this as `src/train.py`.

In [18]:
# Example: src/train.py
import torch
from monai.networks.nets import UNet
from monai.transforms import Compose, LoadImage, EnsureChannelFirst, Resize, ScaleIntensity, ToTensor
from monai.data import DataLoader, Dataset
from monai.losses import DiceLoss
from monai.metrics import DiceMetric
import os
import json

# Load configuration
with open("../config.json") as f:
    config = json.load(f)

# Set device
device = torch.device(config["device"] if torch.cuda.is_available() else "cpu")

# Define transformations
train_transforms = Compose([
    LoadImage(image_only=True),
    EnsureChannelFirst(),
    Resize(config["image_size"]),
    ScaleIntensity(),
    ToTensor()
])

# Define dataset and dataloaders
train_dataset = Dataset(data=[], transform=train_transforms)  # Replace `data=[]` with your dataset
train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True, num_workers=config["num_workers"])

# Define model, loss, and optimizer
model = UNet(
    dimensions=3,
    in_channels=1,
    out_channels=2,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
).to(device)

loss_function = DiceLoss(to_onehot_y=True, softmax=True)
optimizer = torch.optim.Adam(model.parameters(), lr=config["learning_rate"])

# Training loop
for epoch in range(config["epochs"]):
    model.train()
    epoch_loss = 0
    for batch in train_loader:
        inputs, labels = batch["image"].to(device), batch["label"].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch + 1}/{config['epochs']}, Loss: {epoch_loss / len(train_loader)}")

ModuleNotFoundError: No module named 'torch'

## 7. Open in VS Code

Open the project directory in VS Code and install the Python extension for best experience.

In [19]:
# Open the project in VS Code (run in terminal)
!code tumor-detection-segmentation